## Website
https://www.hktvmall.com/hktv/zh/homenfamily

## Environment and Imports
No need to run the cell below if you have already installed the packages

In [1]:
!pip install pyppeteer

In [2]:
from pyppeteer import launch
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup as bs

In [33]:
nest_asyncio.apply()

In [43]:
def run_asyncio(func, args=None):
    return asyncio.get_event_loop().run_until_complete(func(args))

## Getting Pages

In [4]:
TABLET = 0
LAPTOP = 1
PHONE = 2
urls = [
    "https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32301500001",
    "https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32300500001", 
    "https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32201010001"
]

In [32]:
async def get_item_page(url: str):
    '''
    get the web page
    params:
        url: str, url of the web page
    return: 
        page: pyppeteer.page.Page, the web page
    '''
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    return page

In [44]:
category_pages = []
for url in urls:
    page = run_asyncio(get_item_page, url)
    category_pages.append(page)
    print(asyncio.run(page.title()))

平板電腦 | 香港電視 HKTVmall 網上購物
手提電腦 | 香港電視 HKTVmall 網上購物
智能電話 | 香港電視 HKTVmall 網上購物


In [ ]:
async def get_page_num(page):
    